# Tutorial 02: Neuron Morphology

**Author:** Alexander Bates  
**Python Version with navis**

---

## Introduction

The purpose of this tutorial is to examine neuron morphology using the **navis** Python library.

We can visualize neurons, compare their morphological similarity with NBLAST, and analyze their spatial properties.

This tutorial demonstrates:
- Loading neuron skeletons (.swc files) from GCS with navis
- 3D visualization with plotly
- Morphometric analysis (cable length, branch points, etc.)
- Visualizing neurons with neuropil meshes
- NBLAST morphological similarity analysis
- Template brain transformations with flybrains

---

## About navis

[**navis**](https://navis.readthedocs.io/) is a Python library for analyzing and visualizing neuron morphology and connectivity. It provides:

- Support for multiple neuron representations (skeletons, meshes, point clouds)
- Fast NBLAST implementation for morphological comparison
- Template brain transformations via [**flybrains**](https://github.com/navis-org/navis-flybrains)
- Multiple 3D visualization backends (plotly, octarine, k3d)
- Seamless integration with other neuroinformatics tools

navis is the Python equivalent of the R natverse ecosystem.

---

## Setup and Configuration

In [1]:
# Dataset configuration
# Options: "banc_746", "fafb_783", "manc_121", "hemibrain_121", "malecns_09"
DATASET = "banc_746"
DATASET_ID = "banc_746_id"

# We'll focus on a specific neuropil subset to keep examples manageable
SUBSET_NAME = "front_leg"
NEUROPIL_PATTERN = r"^LegNp\(T1\)|T1|^ProNM-T1|^LNp_T3"

# Data location - can be GCS bucket or local path
DATA_PATH = "gs://brain-and-nerve-cord_exports/sjcabs_data"

# Detect if using GCS or local path
USE_GCS = DATA_PATH.startswith("gs://")

# Image output directory
import os
IMG_DIR = "images/tutorial_02"
os.makedirs(IMG_DIR, exist_ok=True)

print(f"Dataset: {DATASET}")
print(f"Subset: {SUBSET_NAME}")
print(f"Data location: {DATA_PATH}")
print(f"Using GCS: {USE_GCS}")
print(f"Images will be saved to: {IMG_DIR}")

Dataset: banc_746
Subset: front_leg
Data location: gs://brain-and-nerve-cord_exports/sjcabs_data
Using GCS: True
Images will be saved to: images/tutorial_02


## Import Packages

We'll use:
- **navis**: Neuron analysis and visualization
- **pandas**: Data manipulation
- **pyarrow**: Reading Feather files
- **gcsfs**: Google Cloud Storage access
- **plotly**: Interactive 3D visualizations
- **trimesh**: 3D mesh processing

**Installation:**
```bash
pip install navis[all] flybrains gcsfs plotly kaleido trimesh
```

In [2]:
import navis
import pandas as pd
import numpy as np
import pyarrow.feather as feather
import gcsfs
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import trimesh
import re
from pathlib import Path
from tqdm.auto import tqdm
import tempfile
import io

# Disable navis progress bars for nbconvert execution
# (avoids ipywidgets requirement)
navis.config.pbar_hide = True

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

print("✓ Packages loaded successfully")
print(f"navis version: {navis.__version__}")
print(f"pandas version: {pd.__version__}")

✓ Packages loaded successfully
navis version: 1.10.0
pandas version: 2.3.3


## Setup GCS Access

**Authentication required:** Before running with GCS, authenticate:

```bash
gcloud auth application-default login
```

In [3]:
# Setup GCS filesystem if needed
if USE_GCS:
    print("Setting up Google Cloud Storage access...")
    gcs = gcsfs.GCSFileSystem(token='google_default')
    print("✓ GCS filesystem initialized")
else:
    gcs = None
    print("Using local filesystem")

Setting up Google Cloud Storage access...
✓ GCS filesystem initialized


## Helper Functions

In [4]:
def construct_path(data_root, dataset, file_type="meta", space_suffix=None):
    """
    Construct file paths for dataset files.
    
    Parameters
    ----------
    data_root : str
        Root data directory (can be gs:// or local path)
    dataset : str
        Dataset name with version (e.g., "banc_746")
    file_type : str
        Type of file: "meta", "synapses", "skeletons"
    space_suffix : str, optional
        Space name for skeletons (defaults to native space)
    
    Returns
    -------
    str
        Full path to the file
    """
    dataset_name = dataset.split("_")[0]
    
    extensions = {
        "meta": ".feather",
        "synapses": ".parquet",
        "skeletons": ""  # No extension - it's a directory
    }
    
    if file_type not in extensions:
        raise ValueError(f"Unknown file_type: {file_type}")
    
    extension = extensions[file_type]
    
    if file_type == "skeletons":
        # Skeleton directories don't include version number
        if space_suffix is None:
            space_suffix = f"{dataset_name}_space"
        
        # BANC uses l2 skeletons
        if dataset_name == "banc":
            filename = f"{dataset_name}_{space_suffix}_l2_swc{extension}"
        else:
            filename = f"{dataset_name}_{space_suffix}_swc{extension}"
    else:
        filename = f"{dataset}_{file_type}{extension}"
    
    full_path = f"{data_root}/{dataset_name}/{filename}"
    return full_path


def read_feather_gcs(path, gcs_fs=None):
    """Read Feather file from GCS or local path."""
    if path.startswith("gs://"):
        if gcs_fs is None:
            raise ValueError("gcs_fs required for GCS paths")
        
        print(f"Reading from GCS: {path}")
        gcs_path = path.replace("gs://", "")
        
        with gcs_fs.open(gcs_path, 'rb') as f:
            df = feather.read_feather(f)
        
        print(f"✓ Loaded {len(df):,} rows")
        return df
    else:
        print(f"Reading from local path: {path}")
        df = pd.read_feather(path)
        print(f"✓ Loaded {len(df):,} rows")
        return df


def read_swc_from_gcs(gcs_fs, swc_path):
    """
    Read a single SWC file from GCS using navis.
    
    Parameters
    ----------
    gcs_fs : gcsfs.GCSFileSystem
        GCS filesystem object
    swc_path : str
        GCS path to SWC file (without gs:// prefix)
    
    Returns
    -------
    navis.TreeNeuron
        Loaded neuron
    """
    # Read file content from GCS
    with gcs_fs.open(swc_path, 'rb') as f:
        content = f.read()
    
    # Create a file-like object from bytes
    swc_file = io.BytesIO(content)
    
    # Read with navis
    neuron = navis.read_swc(swc_file)
    
    return neuron


def batch_read_swc_from_gcs(gcs_fs, directory, filenames, show_progress=True):
    """
    Batch read multiple SWC files from GCS.
    
    Parameters
    ----------
    gcs_fs : gcsfs.GCSFileSystem
        GCS filesystem object
    directory : str
        GCS directory path (without gs:// prefix)
    filenames : list of str
        List of SWC filenames to read
    show_progress : bool
        Whether to show progress bar
    
    Returns
    -------
    navis.NeuronList
        List of loaded neurons
    """
    neurons = []
    
    iterator = tqdm(filenames) if show_progress else filenames
    
    for filename in iterator:
        swc_path = f"{directory}/{filename}"
        try:
            neuron = read_swc_from_gcs(gcs_fs, swc_path)
            neurons.append(neuron)
        except Exception as e:
            print(f"Error reading {filename}: {e}")
            continue
    
    return navis.NeuronList(neurons)


def read_obj_from_gcs(gcs_fs, obj_path):
    """
    Read OBJ mesh file from GCS.
    
    Parameters
    ----------
    gcs_fs : gcsfs.GCSFileSystem
        GCS filesystem object
    obj_path : str
        GCS path to OBJ file (without gs:// prefix)
    
    Returns
    -------
    trimesh.Trimesh
        Loaded mesh
    """
    with gcs_fs.open(obj_path, 'rb') as f:
        content = f.read()
    
    # Create temporary file for trimesh
    with tempfile.NamedTemporaryFile(suffix='.obj', delete=False) as tmp:
        tmp.write(content)
        tmp_path = tmp.name
    
    try:
        mesh = trimesh.load_mesh(tmp_path)
    finally:
        os.unlink(tmp_path)
    
    return mesh


print("✓ Helper functions defined")

✓ Helper functions defined


## Setup File Paths

In [5]:
# Construct paths
meta_path = construct_path(DATA_PATH, DATASET, "meta")
skeletons_path = construct_path(DATA_PATH, DATASET, "skeletons")

# Extract base dataset name
dataset_base = DATASET.split("_")[0]

print("File paths:")
print(f"  Metadata: {meta_path}")
print(f"  Skeletons: {skeletons_path}")

File paths:
  Metadata: gs://brain-and-nerve-cord_exports/sjcabs_data/banc/banc_746_meta.feather
  Skeletons: gs://brain-and-nerve-cord_exports/sjcabs_data/banc/banc_banc_space_l2_swc


---

## Read Meta Data

In [6]:
# Load metadata
meta = read_feather_gcs(meta_path, gcs_fs=gcs)

print(f"\nTotal neurons: {len(meta):,}")
meta.head()

Reading from GCS: gs://brain-and-nerve-cord_exports/sjcabs_data/banc/banc_746_meta.feather


✓ Loaded 168,793 rows

Total neurons: 168,793


,banc_746_id,supervoxel_id,region,side,hemilineage,nerve,flow,super_class,cell_class,cell_sub_class,cell_type,neurotransmitter_predicted,neurotransmitter_score,cell_function,cell_function_detailed,body_part_sensory,body_part_effector,status
0,720575941569192238,74803281603754231,central_brain,right,VPNp1_medial,None,intrinsic,central_brain_intrinsic,None,None,"(PLP191,PLP192)a",acetylcholine,0.7534,None,None,None,None,
1,720575941574697871,74873512908765054,central_brain,right,VPNp1_medial,None,intrinsic,central_brain_intrinsic,None,None,"(PLP191,PLP192)a",acetylcholine,0.7976,None,None,None,None,
2,720575941652939029,77477362601861709,central_brain,left,VPNp1_medial,None,intrinsic,central_brain_intrinsic,None,None,"(PLP191,PLP192)a",dopamine,0.5825,None,None,None,None,TRACING_ISSUE_2
3,720575941452014202,74310563223910394,central_brain,right,VPNp1_medial,None,intrinsic,central_brain_intrinsic,None,None,"(PLP191,PLP192)a",acetylcholine,0.5704,None,None,None,None,
4,720575941565035527,77406993858043384,central_brain,left,VPNp1_medial,None,intrinsic,central_brain_intrinsic,None,None,"(PLP191,PLP192)a",acetylcholine,0.6317,None,None,None,None,


### Explore the Dataset

Let's examine the available neurons in this dataset:

In [7]:
# For this tutorial, we'll work with DNg12 neurons
# Instead of filtering by neuropil (which requires subset files),
# we'll filter directly by cell type when we need specific neurons

print(f"Total neurons in dataset: {len(meta):,}")
print(f"\nWe'll focus on DNg12 neurons for morphology analysis")

# No pre-filtering needed - we'll select neurons by cell_type as needed
meta.head()

Total neurons in dataset: 168,793

We'll focus on DNg12 neurons for morphology analysis


,banc_746_id,supervoxel_id,region,side,hemilineage,nerve,flow,super_class,cell_class,cell_sub_class,cell_type,neurotransmitter_predicted,neurotransmitter_score,cell_function,cell_function_detailed,body_part_sensory,body_part_effector,status
0,720575941569192238,74803281603754231,central_brain,right,VPNp1_medial,None,intrinsic,central_brain_intrinsic,None,None,"(PLP191,PLP192)a",acetylcholine,0.7534,None,None,None,None,
1,720575941574697871,74873512908765054,central_brain,right,VPNp1_medial,None,intrinsic,central_brain_intrinsic,None,None,"(PLP191,PLP192)a",acetylcholine,0.7976,None,None,None,None,
2,720575941652939029,77477362601861709,central_brain,left,VPNp1_medial,None,intrinsic,central_brain_intrinsic,None,None,"(PLP191,PLP192)a",dopamine,0.5825,None,None,None,None,TRACING_ISSUE_2
3,720575941452014202,74310563223910394,central_brain,right,VPNp1_medial,None,intrinsic,central_brain_intrinsic,None,None,"(PLP191,PLP192)a",acetylcholine,0.5704,None,None,None,None,
4,720575941565035527,77406993858043384,central_brain,left,VPNp1_medial,None,intrinsic,central_brain_intrinsic,None,None,"(PLP191,PLP192)a",acetylcholine,0.6317,None,None,None,None,


### Examine Cell Types

What are the top cell types in this dataset?

In [8]:
# Count by cell type
cell_type_counts = meta['cell_type'].value_counts().head(15)

print("Top 15 cell types in dataset:")
print(cell_type_counts)

# Create bar plot
fig = go.Figure()

fig.add_trace(go.Bar(
    x=cell_type_counts.index,
    y=cell_type_counts.values,
    marker_color='steelblue',
    text=cell_type_counts.values,
    textposition='outside'
))

fig.update_layout(
    title=f"Top Cell Types: {DATASET}",
    xaxis_title="Cell Type",
    yaxis_title="Count",
    template="plotly_white",
    height=500,
    xaxis_tickangle=-45
)

fig.show()

Top 15 cell types in dataset:
cell_type
KCg-m      1597
KCab       1261
BM_InOm    1023
Tm3         981
T3          919
Mi1         879
L5          878
L1          877
Tm1         846
R7          840
R8          828
L4          820
T4c         810
T5a         810
T4d         809
Name: count, dtype: int64


---

## Read Neuron Skeletons (.swc files)

Neuron skeletons are stored as SWC files in the GCS bucket. We'll use navis to read them.

### Load a Single Neuron

Let's start by loading one DNg12 neuron:

In [9]:
# Filter for DNg12 neurons from full metadata
meta_dng12 = meta[
    meta['cell_type'].str.contains('DNg12', case=False, na=False)
]

if len(meta_dng12) > 0:
    selected_id = meta_dng12[DATASET_ID].iloc[0]
    print(f"Found {len(meta_dng12)} DNg12 neurons")
    print(f"Selected neuron ID: {selected_id}")
    
    # Construct path to SWC file
    swc_filename = f"{selected_id}.swc"
    
    # Read neuron from GCS
    if USE_GCS:
        gcs_skeleton_path = skeletons_path.replace("gs://", "")
        print(f"Reading from GCS: {gcs_skeleton_path}/{swc_filename}")
        neuron = read_swc_from_gcs(gcs, f"{gcs_skeleton_path}/{swc_filename}")
    else:
        neuron = navis.read_swc(f"{skeletons_path}/{swc_filename}")
    
    print(f"\n✓ Loaded neuron with {neuron.n_nodes:,} nodes")
    print(f"  Cable length: {neuron.cable_length:,.0f} nm")
    print(f"  Number of branches: {neuron.n_branches}")
    print(f"  Number of leaf nodes: {neuron.n_leafs}")
else:
    print("No DNg12 neurons found in this dataset")

Found 46 DNg12 neurons
Selected neuron ID: 720575941573370696
Reading from GCS: brain-and-nerve-cord_exports/sjcabs_data/banc/banc_banc_space_l2_swc/720575941573370696.swc

✓ Loaded neuron with 260 nodes
  Cable length: 1,171,886 nm
  Number of branches: 57
  Number of leaf nodes: 90


### 3D Visualization with Plotly

navis makes it easy to visualize neurons in 3D with plotly backend:

In [10]:
if len(meta_dng12) > 0:
    # Convert from nm to µm for better scale
    neuron_um = neuron / 1000
    neuron_um.units = 'µm'  # Update units metadata
    
    # Plot with plotly
    # Note: navis.plot3d with plotly backend shows the plot automatically
    navis.plot3d(
        neuron_um,
        backend='plotly',
        color='darkblue',
        width=1200,
        height=800,
        title=f"DNg12 Neuron: {selected_id}"
    )

---

## Read Neuropil Meshes

Neuropil meshes are stored as OBJ files. We can load them and visualize neurons in their anatomical context.

In [11]:
# Construct path to neuropil meshes
obj_path = f"{DATA_PATH}/{dataset_base}/obj/neuropils"
print(f"Neuropil meshes path: {obj_path}")

# List available OBJ files
if USE_GCS:
    gcs_obj_path = obj_path.replace("gs://", "")
    obj_files = gcs.ls(gcs_obj_path)
    obj_files = [f"gs://{f}" for f in obj_files if f.endswith('.obj')]
else:
    import glob
    obj_files = glob.glob(f"{obj_path}/*.obj")

print(f"\nFound {len(obj_files)} OBJ files")
if obj_files:
    print("Example files:")
    for f in obj_files[:3]:
        print(f"  {f}")

Neuropil meshes path: gs://brain-and-nerve-cord_exports/sjcabs_data/banc/obj/neuropils



Found 112 OBJ files
Example files:
  gs://brain-and-nerve-cord_exports/sjcabs_data/banc/obj/neuropils/banc_neuropil_COURT_vnc_ABDNM_nm.obj
  gs://brain-and-nerve-cord_exports/sjcabs_data/banc/obj/neuropils/banc_neuropil_COURT_vnc_AMNP_nm.obj
  gs://brain-and-nerve-cord_exports/sjcabs_data/banc/obj/neuropils/banc_neuropil_COURT_vnc_HTct_nm.obj


### Load and Visualize Multiple Neurons with Neuropil Context

Let's load VNC and leg neuropil meshes and visualize multiple DNg12 neurons in their anatomical context, each in a different color:

In [12]:
if len(meta_dng12) > 0 and obj_files:
    # Find VNC and leg neuropil meshes
    vnc_meshes = [f for f in obj_files if 'vnc' in f.lower()]
    leg_meshes = [f for f in obj_files if re.search(NEUROPIL_PATTERN, f, re.IGNORECASE)]
    
    meshes_to_load = []
    if vnc_meshes:
        meshes_to_load.append(vnc_meshes[0])
    if leg_meshes:
        meshes_to_load.append(leg_meshes[0])
    
    if meshes_to_load:
        print(f"Loading {len(meshes_to_load)} mesh(es)...")
        
        loaded_meshes = []
        for mesh_file in meshes_to_load:
            print(f"  {mesh_file}")
            if USE_GCS:
                gcs_mesh_path = mesh_file.replace("gs://", "")
                mesh = read_obj_from_gcs(gcs, gcs_mesh_path)
            else:
                mesh = trimesh.load_mesh(mesh_file)
            
            # Convert to µm to match neuron
            mesh.vertices = mesh.vertices / 1000
            loaded_meshes.append(mesh)
        
        print("\n✓ Meshes loaded")
        
        # Load first 5 DNg12 neurons for visualization
        n_vis = min(5, len(meta_dng12))
        vis_ids = meta_dng12[DATASET_ID].iloc[:n_vis].values
        
        print(f"\nLoading {n_vis} DNg12 neurons for visualization...")
        vis_filenames = [f"{nid}.swc" for nid in vis_ids]
        
        if USE_GCS:
            gcs_skeleton_path = skeletons_path.replace("gs://", "")
            vis_neurons = batch_read_swc_from_gcs(gcs, gcs_skeleton_path, vis_filenames, show_progress=False)
        else:
            vis_neurons = []
            for fname in vis_filenames:
                try:
                    n = navis.read_swc(f"{skeletons_path}/{fname}")
                    vis_neurons.append(n)
                except Exception as e:
                    print(f"Error reading {fname}: {e}")
            vis_neurons = navis.NeuronList(vis_neurons)
        
        # Convert to µm
        vis_neurons_um = vis_neurons / 1000
        
        print(f"✓ Loaded {len(vis_neurons_um)} neurons")
        
        # Create visualization with plotly
        fig = go.Figure()
        
        # Add meshes
        mesh_colors = ['lightgrey', 'lightblue']
        mesh_alphas = [0.1, 0.3]
        mesh_names = ['VNC', 'Leg Neuropil']
        
        for i, mesh in enumerate(loaded_meshes):
            fig.add_trace(go.Mesh3d(
                x=mesh.vertices[:, 0],
                y=mesh.vertices[:, 1],
                z=mesh.vertices[:, 2],
                i=mesh.faces[:, 0],
                j=mesh.faces[:, 1],
                k=mesh.faces[:, 2],
                color=mesh_colors[i % len(mesh_colors)],
                opacity=mesh_alphas[i % len(mesh_alphas)],
                name=mesh_names[i % len(mesh_names)],
                hoverinfo='name'
            ))
        
        # Add neurons in different colors
        neuron_colors = ['darkblue', 'darkred', 'darkgreen', 'darkorange', 'purple']
        
        for i, neuron in enumerate(vis_neurons_um):
            neuron_trace = neuron.nodes
            fig.add_trace(go.Scatter3d(
                x=neuron_trace['x'],
                y=neuron_trace['y'],
                z=neuron_trace['z'],
                mode='lines',
                line=dict(color=neuron_colors[i % len(neuron_colors)], width=3),
                name=f'DNg12 Neuron {i+1}',
                hoverinfo='name'
            ))
        
        fig.update_layout(
            title=f"{len(vis_neurons_um)} DNg12 Neurons in Neuropil Context: {DATASET}",
            scene=dict(
                xaxis_title='X (µm)',
                yaxis_title='Y (µm)',
                zaxis_title='Z (µm)',
                aspectmode='data'
            ),
            width=1200,
            height=800
        )
        
        fig.show()
    else:
        print("No matching neuropil meshes found")
else:
    print("Skipping neuropil visualization (no DNg12 neurons or meshes available)")

Loading 2 mesh(es)...
  gs://brain-and-nerve-cord_exports/sjcabs_data/banc/obj/neuropils/banc_neuropil_COURT_vnc_ABDNM_nm.obj


/opt/miniconda3/envs/sjcabs/lib/python3.10/site-packages/trimesh/grouping.py:99: RuntimeWarning:

invalid value encountered in cast



  gs://brain-and-nerve-cord_exports/sjcabs_data/banc/obj/neuropils/banc_neuropil_COURT_vnc_ProNM-T1_nm.obj



✓ Meshes loaded

Loading 5 DNg12 neurons for visualization...


✓ Loaded 5 neurons


---

## Load Multiple Neurons for NBLAST

Let's load neurons from the ventral nerve cord for morphological comparison. Following the R tutorial approach, we'll filter for intrinsic VNC neurons restricted to a single leg neuromere, and take one example per cell type for computational efficiency:

In [13]:
# Filter for intrinsic VNC neurons in single leg neuromere
# (Same filtering approach as R tutorial for consistency)
meta_vnc = meta[
    (meta['super_class'] == 'ventral_nerve_cord_intrinsic') &
    (meta['cell_class'] == 'single_leg_neuromere')
].copy()

print(f"Found {len(meta_vnc)} VNC intrinsic neurons in single leg neuromere")

# Take one neuron per cell type for speed (as done in R tutorial)
# Note: This is just for tutorial performance, not usually done in real analysis
meta_vnc_sampled = meta_vnc.drop_duplicates(subset='cell_type', keep='first')

print(f"Selected {len(meta_vnc_sampled)} neurons (one per cell type)")
print(f"\nTop cell types:")
print(meta_vnc_sampled['cell_type'].value_counts().head(10))

# Get neuron IDs
sampled_ids = meta_vnc_sampled[DATASET_ID].values

if len(sampled_ids) > 0:
    print(f"\nLoading {len(sampled_ids)} neurons for NBLAST analysis...")
    
    # Prepare filenames
    swc_filenames = [f"{nid}.swc" for nid in sampled_ids]
    
    # Batch read from GCS
    if USE_GCS:
        gcs_skeleton_path = skeletons_path.replace("gs://", "")
        neurons = batch_read_swc_from_gcs(gcs, gcs_skeleton_path, swc_filenames, show_progress=True)
    else:
        # Local reading
        neurons = []
        for fname in tqdm(swc_filenames):
            try:
                n = navis.read_swc(f"{skeletons_path}/{fname}")
                neurons.append(n)
            except Exception as e:
                print(f"Error reading {fname}: {e}")
        neurons = navis.NeuronList(neurons)
    
    print(f"\n✓ Loaded {len(neurons)} neurons successfully")
else:
    print("No neurons found matching the filter criteria")
    neurons = navis.NeuronList([])

Found 2844 VNC intrinsic neurons in single leg neuromere
Selected 424 neurons (one per cell type)

Top cell types:
cell_type
CV1005      1
IN16B032    1
IN16B029    1
IN16B022    1
IN16B020    1
IN16B018    1
IN16B016    1
IN14B010    1
IN14B005    1
IN14A120    1
Name: count, dtype: int64

Loading 424 neurons for NBLAST analysis...


  0%|          | 0/424 [00:00<?, ?it/s]

Error reading 720575941443204559.swc: b/brain-and-nerve-cord_exports/o/sjcabs_data%2Fbanc%2Fbanc_banc_space_l2_swc%2F720575941443204559.swc



✓ Loaded 423 neurons successfully


### Morphometric Analysis

Calculate morphological properties for all neurons:

In [14]:
# Convert to microns
neurons_um = neurons / 1000

# Extract morphological properties
morpho_df = pd.DataFrame({
    'neuron_id': sampled_ids[:len(neurons_um)],
    'cell_type': meta_vnc_sampled['cell_type'].values[:len(neurons_um)],
    'cable_length_um': neurons_um.cable_length,
    'n_nodes': [n.n_nodes for n in neurons_um],
    'n_branches': [n.n_branches for n in neurons_um],
    'n_leafs': [n.n_leafs for n in neurons_um]
})

print("\nMorphological Summary:")
print(morpho_df.describe())

morpho_df.head(10)


Morphological Summary:
       cable_length_um      n_nodes  n_branches      n_leafs
count       423.000000   423.000000  423.000000   423.000000
mean       2391.272949   622.609929  134.638298   353.054374
std        1742.591797   469.846583   98.242394   294.466325
min         402.057770    92.000000   15.000000    29.000000
25%        1145.862061   297.000000   65.500000   150.000000
50%        1689.441895   431.000000   97.000000   234.000000
75%        3143.325928   797.500000  178.500000   450.000000
max        9477.973633  2631.000000  552.000000  1660.000000


,neuron_id,cell_type,cable_length_um,n_nodes,n_branches,n_leafs
0,720575941431932257,CV1005,2250.989990,600,140,317
1,720575941723204666,IN00A009,2938.617188,765,167,425
2,720575941558953231,IN00A014,2427.377441,616,125,368
3,720575941438483218,IN00A019,2253.318604,633,127,392
4,720575941630133798,IN01A012,4496.437012,1193,248,694
5,720575941609673802,IN01A015,3724.791016,924,199,542
6,720575941526752356,IN01A025,2679.107910,717,157,397
7,720575941617573845,IN01A030,2267.832520,529,110,275
8,720575941531480805,IN01A032,2444.033203,622,134,318
9,720575941513795148,IN01A034,3176.564941,789,170,417


### Visualize Morphological Distributions

In [15]:
# Create subplots for different metrics
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Cable Length', 'Number of Nodes', 'Number of Branches', 'Number of Leaf Nodes'),
    specs=[[{"type": "histogram"}, {"type": "histogram"}],
           [{"type": "histogram"}, {"type": "histogram"}]]
)

# Cable length
fig.add_trace(
    go.Histogram(x=morpho_df['cable_length_um'], nbinsx=20, marker_color='steelblue', name='Cable Length'),
    row=1, col=1
)

# Number of nodes
fig.add_trace(
    go.Histogram(x=morpho_df['n_nodes'], nbinsx=20, marker_color='seagreen', name='Nodes'),
    row=1, col=2
)

# Number of branches
fig.add_trace(
    go.Histogram(x=morpho_df['n_branches'], nbinsx=20, marker_color='coral', name='Branches'),
    row=2, col=1
)

# Number of leafs
fig.add_trace(
    go.Histogram(x=morpho_df['n_leafs'], nbinsx=20, marker_color='mediumpurple', name='Leaf Nodes'),
    row=2, col=2
)

fig.update_xaxes(title_text="Cable Length (µm)", row=1, col=1)
fig.update_xaxes(title_text="Number of Nodes", row=1, col=2)
fig.update_xaxes(title_text="Number of Branches", row=2, col=1)
fig.update_xaxes(title_text="Number of Leaf Nodes", row=2, col=2)

fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Count", row=1, col=2)
fig.update_yaxes(title_text="Count", row=2, col=1)
fig.update_yaxes(title_text="Count", row=2, col=2)

fig.update_layout(
    title_text=f"Morphological Distributions: {DATASET} - VNC Intrinsic Neurons",
    showlegend=False,
    height=800,
    template="plotly_white"
)

fig.show()

---

## NBLAST Morphological Similarity

**NBLAST** (Neuron BLAST) is an algorithm for comparing neuron morphology based on local geometry. It works by:

1. Converting neurons to point clouds with tangent vectors (dotprops)
2. For each point in query neuron, finding nearest point in target
3. Computing similarity based on distance and angle between tangent vectors

navis includes a fast NBLAST implementation (uses compiled Rust code under the hood).

**Note:** For this tutorial, we'll demonstrate the code but not execute it since it requires additional computation time. Remove `eval=False` to run it yourself.

In [16]:
# Convert neurons to dotprops (point clouds with tangent vectors)
# This removes connectivity but preserves spatial structure
print("Converting neurons to dotprops representation...")
neurons_dp = navis.make_dotprops(neurons_um, k=5)

print("✓ Converted to dotprops")
print(f"  {len(neurons_dp)} neurons ready for NBLAST")

# Run NBLAST all-by-all comparison
# When query and target are the same, performs all-by-all
print("\nRunning NBLAST (this may take a moment)...")
nblast_scores = navis.nblast(neurons_dp, neurons_dp, n_cores=4)

print("✓ NBLAST complete!")
print(f"Score matrix shape: {nblast_scores.shape}")

# Display score matrix
nblast_scores.head()

Converting neurons to dotprops representation...


✓ Converted to dotprops
  423 neurons ready for NBLAST

Running NBLAST (this may take a moment)...


Preparing:   0%|          | 0/1 [00:00<?, ?it/s]

NBlasting:   0%|          | 0/423 [00:00<?, ?it/s]

✓ NBLAST complete!
Score matrix shape: (423, 423)


target,e8620ea1-8e68-4094-93c8-9ee425921d5f,93e82e5a-55eb-4c25-a74f-4256afe1d070,13ca8102-353d-4887-8196-7a8a833a886e,d3bad1b0-933f-42c0-ae44-cb95b8264708,3fe6c6f2-20ec-4832-8f39-7a242844665b,6621968a-bcc3-419c-aaca-d8c16c839012,0a6ad2ca-13c7-40cc-a937-fab50d1a18cb,5f1ecb47-17da-4d94-9a20-e7732e3a845e,e0d94a5d-3a27-40f0-a7b2-341b181949c0,5e7e2aed-e2f9-41fe-b0a3-174027beaebf,0f224433-1941-45eb-a58d-037c749df7df,544936ac-b0c9-4711-b703-796040f9c971,5e053ce2-1307-41da-8167-370d00d609cf,8ee416ae-568b-4bf2-9b4e-c5d052665ce5,2c58bf1d-938f-4c0d-a765-540008546dad,67e6f8bd-3ec7-4a9a-9418-bb621db86d8b,c177f8c1-662c-4e82-86c1-fc2057911d67,41519513-4ffd-482f-b5e3-bd024ce100f1,0220ae30-5049-422e-80bf-89b42af64797,8b3b0e06-c1f8-455b-bee3-4087cb1f3114,e59d8d8f-b0cf-45cc-8721-71728489950b,f9bd72b8-a65d-4c12-8a75-b2cbbf62e1aa,a08caa95-55e2-4e4d-941d-8805cc79e844,f710f03f-a493-4937-85ad-1308ad081f0e,8c252d83-a9ae-4933-a712-47ab337d7737,74f152d6-6fcc-4446-9050-f8288bc9f7a1,9f521711-e0bd-4bb3-8ada-abb1662dd5d8,8e1a0b44-0516-4898-a6fd-407e2ddf6ba7,889654fa-1270-418b-b0f0-a4b165338826,84ded111-8f86-46e4-8a02-37a209614593,b3869a70-64f3-4ba3-8828-6ceee7d9df50,f6be6a8b-0107-496e-9a66-595e062fbc4f,3002a5bd-e83d-44c6-9067-9609872f5ee9,00cac935-587a-4c4d-8ba0-3d258eab0f16,6c8712eb-2c95-456a-8b3d-68fd4908a4cb,64c9b86d-a341-43a2-857b-e974f4729054,d6a4ec7f-fc35-40c9-85f5-0ae1312e2074,317e72e7-51aa-48ac-9cdf-8122eb48c1a4,a5d19615-f325-462a-a679-a347934c5db0,38edb781-a066-499d-b70a-149442226fd5,894e8e63-d828-4087-a3e8-279732c6ac4e,16335bf8-2039-4729-a901-4f11aa2a2898,c46f2252-7024-4d3a-a11d-76060e4d1e10,d29a1c3f-fa5d-44a6-8de5-583db3f97718,f1125c9f-55df-4b83-b499-f15123bc875c,503166a9-6b84-4929-ac40-d6290048ca22,54c0f6a5-25b6-4b07-962a-bbaef127ea52,d2a24e96-335d-48f3-b930-f06e5588548d,62fa817d-3c8e-4aa1-82e5-66feb9b9a5d4,5cb23d93-8775-46bd-8afe-daf494ecd873,92c68a7d-f9b7-45b5-bb19-9afde677ef20,3f112d98-66b0-4612-a615-5af6b0c84be4,beb1789a-b632-40b7-ba69-5c52fcfd4dc1,007faad0-862b-4b0c-a4b5-862a2f61c5c2,573665a4-13a1-4dff-be78-1a04162d3263,667f357b-6985-4e10-857e-77ec47076982,04020e32-a420-436e-86ea-a74cf25ce304,8b6916f0-adab-4e40-9bff-8098c0a1fc1d,5f5ae5d4-2678-486d-a28e-ee69fdb5e901,284f6e7f-81d3-474d-8df0-2f5ebd854856,02d67ad4-1376-4a2e-8731-5ab708af3f98,052df02d-2568-4ea1-b9a1-bd8265834f5d,416b1224-6ef1-4669-b0b5-47b62e98eca1,637abebd-d42d-414b-973e-a21343f6b520,2165535b-a1df-441d-9602-1425086a25d2,9f12c570-0d9a-4062-bbf4-a8449907e97d,74b078ad-01ba-4f38-a9ab-f4a6d92d7a5e,c41db747-5a76-444a-a5ef-db16f9a8f50e,d7ad61cf-8c0d-4304-8cc1-ed5166441b50,33b298ac-354b-49e7-b607-7159254e65da,4b1ff79a-c938-455f-8668-7680a163686c,e3db3a6e-d171-4a89-9dce-f28140674667,418d861d-904d-4211-9560-fe5bf4b578b5,bdc93fd4-95ea-4408-8a3f-29ce61d49f72,c67a6f20-b7a0-4f4d-8244-01b0cc2d5d37,a2e51558-6d88-4083-873d-a7ea16388768,d4f51744-c704-4471-888d-4d5e4f0192c5,dbacc586-c07a-4f00-bfee-f281a97cff67,8d639368-02b2-4c82-89b8-4b10df259a61,50d1d2a9-154a-436f-ab1d-f1e7479c75a6,1e7f327f-e4aa-4532-80a1-b46367e5e1d6,4d1f05f2-e8ee-4fff-80dc-edbbb5e71eed,7d6d7eaa-29ba-419e-9482-d4196a242fa0,31718a06-b6e1-4095-8083-76ef4384a350,a573fdd4-7e84-4f73-8999-1627cd8a3dc6,1f6d0a98-896f-4b34-9539-3cec32c09542,2cdd2347-b4db-49f1-8706-945233f43005,f8d7c971-6bd5-4664-95fd-6f6c9d0b1577,f2e4fa7e-9855-49e5-abac-3c82774f4bb3,cc73c393-c691-4af9-a8ab-9607ca2153ce,c070457e-ae6f-481d-8009-2f5874e43e9b,b4b66775-e974-49ed-b430-a1ffebfe1ae9,78160529-780d-4e95-af0e-4cc7324d54c4,4cd618e5-5d8f-4edd-992c-990c5932adec,03539383-46ef-4228-9c82-c79edfc20298,81b0b06d-cd3f-4e42-b36f-b384c8cfc691,aaa446f5-f787-4ca6-b2ef-226032bd5e4e,5961672c-96d1-4606-b716-647a8bb565dc,34f0aad4-910a-448e-ba2c-bd139ecfb416,15dc65b7-1cba-4387-a979-6f706c0e005e,bc6e607d-ba3d-4413-8dd5-a47bf4be783a,b1fb3fdc-5379-4528-9eec-4e1f282bd18b,96cf2a73-1ee0-4034-bfc1-ff18de7aa67c,3972511d-0acd-4b0f-a36e-58626d57614d,8b2097de-acf6-48fb-a3c8-03673da43a8e,910501e4-0da9-4515-9b09-b9297a5e1055,c04497f5-8ada-4484-a4c4-a16d36c184f0,9c28c6fb-4fdc-49e8-a30a-95d3084c7c

### Hierarchical Clustering

Use NBLAST scores to cluster neurons by morphological similarity:

In [17]:
from scipy.cluster.hierarchy import linkage, dendrogram, cut_tree
from scipy.spatial.distance import squareform
import plotly.figure_factory as ff

# Convert NBLAST scores to distance matrix
# NBLAST scores are similarity scores (higher = more similar)
# We need to normalize by self-scores and convert to distances
self_scores = np.diag(nblast_scores.values)
normalized_scores = nblast_scores.values / self_scores.max()
distance_matrix = 1 - normalized_scores

# Convert to condensed distance matrix for linkage
# Only use upper triangle (distance matrix is symmetric)
condensed_dist = squareform(distance_matrix, checks=False)

# Perform hierarchical clustering
linkage_matrix = linkage(condensed_dist, method='ward')

# Choose number of clusters
k = 20
print(f"Cutting dendrogram into {k} clusters...")

# Cut tree to get cluster assignments
clusters = cut_tree(linkage_matrix, n_clusters=k).flatten() + 1  # +1 to start from 1

# Get cut height for k clusters
# Height at which we cut is at the (n-k)th merge
cut_height = linkage_matrix[len(linkage_matrix) - k, 2]

print(f"Cut height for k={k}: {cut_height:.4f}")
print(f"\nCluster sizes:")
cluster_counts = pd.Series(clusters).value_counts().sort_index()
print(cluster_counts)

# Create dendrogram with plotly
# First create using scipy to get dendrogram coordinates
dend = dendrogram(linkage_matrix, no_plot=True)

# Get leaf order and corresponding clusters
leaf_order = dend['leaves']
leaf_clusters = clusters[leaf_order]

# Create color map for clusters
import plotly.express as px
colors_list = px.colors.qualitative.Light24 + px.colors.qualitative.Dark24
cluster_colors = {i: colors_list[i % len(colors_list)] for i in range(1, k+1)}
leaf_colors = [cluster_colors[c] for c in leaf_clusters]

# Create figure
fig = go.Figure()

# Add dendrogram segments
for i, (x, y) in enumerate(zip(dend['icoord'], dend['dcoord'])):
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='lines',
        line=dict(color='grey', width=1),
        hoverinfo='skip',
        showlegend=False
    ))

# Add colored points at leaves
leaf_x = [(dend['icoord'][i][1] + dend['icoord'][i][2]) / 2 
          for i in range(len(dend['icoord'])) 
          if dend['dcoord'][i][0] == 0]
leaf_y = [0] * len(leaf_x)

# Group leaves by cluster for legend
for cluster_id in sorted(cluster_counts.index):
    cluster_mask = leaf_clusters == cluster_id
    cluster_x = [x for i, x in enumerate(leaf_x) if cluster_mask[i]]
    cluster_y = [0] * len(cluster_x)
    
    fig.add_trace(go.Scatter(
        x=cluster_x,
        y=cluster_y,
        mode='markers',
        marker=dict(size=8, color=cluster_colors[cluster_id]),
        name=f'Cluster {cluster_id}',
        hovertext=[f'Cluster {cluster_id}'] * len(cluster_x),
        hoverinfo='text'
    ))

# Add horizontal cut line
fig.add_hline(
    y=cut_height,
    line=dict(color='red', width=2, dash='dash'),
    annotation_text=f'Cut height (k={k})',
    annotation_position='right'
)

# Update layout
fig.update_layout(
    title=dict(
        text=f'Hierarchical Clustering of Neuron Morphology (NBLAST)<br><sub>{DATASET} - {SUBSET_NAME} (k={k} clusters)</sub>',
        x=0.5,
        xanchor='center'
    ),
    xaxis=dict(
        title='Neurons',
        showticklabels=False,
        showgrid=False
    ),
    yaxis=dict(
        title='Height (Ward Distance)',
        showgrid=True,
        gridcolor='lightgrey'
    ),
    plot_bgcolor='white',
    width=1200,
    height=600,
    hovermode='closest',
    legend=dict(
        orientation='v',
        yanchor='top',
        y=1,
        xanchor='left',
        x=1.01,
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='grey',
        borderwidth=1
    )
)

# Save static version
fig.write_image(f"{IMG_DIR}/{DATASET}_{SUBSET_NAME}_nblast_dendrogram.png", 
                width=1400, height=700, scale=2)
print(f"\n✓ Dendrogram saved to {IMG_DIR}/{DATASET}_{SUBSET_NAME}_nblast_dendrogram.png")

# Display interactive version
fig.show()

Cutting dendrogram into 20 clusters...
Cut height for k=20: 2.2233

Cluster sizes:
1     12
2     27
3     11
4      7
5     61
6      8
7     14
8     38
9     18
10    13
11    22
12    28
13    24
14    23
15    20
16    18
17    15
18    35
19    26
20     3
Name: count, dtype: int64



✓ Dendrogram saved to images/tutorial_02/banc_746_front_leg_nblast_dendrogram.png


---

## Template Brain Transformations

The [**flybrains**](https://github.com/navis-org/navis-flybrains) package provides template brain transformations for Drosophila connectomes.

It supports 31+ template brains including:
- FAFB14, FLYWIRE
- JRC2018F/M/U (Janelia Reference Brains)
- JRCFIB2022M (maleCNS)
- FANC, BANC, MANC
- VNC templates

**Note:** This requires external dependencies (CMTK or Elastix). See [flybrains documentation](https://github.com/navis-org/navis-flybrains) for installation.

In [18]:
try:
    import flybrains
    
    print("✓ flybrains loaded")
    print(f"  Version: {flybrains.__version__}")
    
    # Download optional transforms (only needs to be done once)
    # Uncomment these if you haven't downloaded them yet:
    # flybrains.download_jefferislab_transforms()
    # flybrains.download_jrc_transforms()
    # flybrains.download_jrc_vnc_transforms()
    # flybrains.register_transforms()
    
    # Check available transforms
    print("\nExample: Transform BANC neurons to JRC2018F space:")
    print("")
    print("```python")
    print("# Transform a single neuron")
    print("neuron_jrc = navis.xform_brain(neuron_um, source='BANC', target='JRC2018F')")
    print("")
    print("# Transform multiple neurons")
    print("neurons_jrc = navis.xform_brain(neurons_um, source='BANC', target='JRC2018F')")
    print("```")
    
except ImportError:
    print("flybrains not installed")
    print("Install with: pip install flybrains")
    print("See: https://github.com/navis-org/navis-flybrains")

flybrains not installed
Install with: pip install flybrains
See: https://github.com/navis-org/navis-flybrains


---

## Summary

In this tutorial, we covered:

1. **Loading Neuron Skeletons**: Reading SWC files from GCS with navis
2. **3D Visualization**: Interactive plotting with plotly backend
3. **Morphometric Analysis**: Extracting cable length, branch counts, etc.
4. **Neuropil Meshes**: Visualizing neurons in anatomical context
5. **NBLAST**: Morphological similarity comparison
6. **Template Brains**: Introduction to flybrains for spatial transformations

### Key navis Features

- **Simple API**: `navis.read_swc()`, `navis.plot3d()`, `navis.nblast()`
- **Fast**: Compiled code (Rust) for NBLAST and other operations
- **Flexible**: Multiple backends (plotly, octarine, k3d)
- **Integrated**: Works with pandas, numpy, and other scientific Python tools
- **Well-documented**: Extensive [documentation](https://navis.readthedocs.io/) and [tutorials](https://navis-org.github.io/navis/)

### Useful navis Functions

```python
# Reading neurons
navis.read_swc('neuron.swc')
navis.read_swc('neurons.zip', include_subdirs=True)

# Morphometrics
n.cable_length  # Total cable length
n.n_branches    # Number of branches
n.n_nodes       # Number of nodes
navis.segment_analysis(n)  # Detailed per-segment metrics

# Visualization
navis.plot3d(neurons, backend='plotly', color='blue')
navis.plot3d(neurons, backend='octarine')  # Modern, fast backend

# Morphological comparison
dotprops = navis.make_dotprops(neurons, k=5)
scores = navis.nblast(dotprops, dotprops)

# Spatial transformations
navis.xform_brain(neuron, source='BANC', target='JRC2018F')

# Manipulation
neurons / 1000  # Scale (e.g., nm to µm)
navis.downsample_neuron(n, factor=5)  # Reduce resolution
navis.prune_by_strahler(n, to_prune=1)  # Prune terminal branches
```

---

## Next Steps

- Explore Tutorial 03 for connectivity analysis
- Try different datasets (FAFB, MANC, etc.)
- Experiment with different NBLAST parameters
- Use flybrains to compare neurons across datasets
- Check out the [navis documentation](https://navis.readthedocs.io/) for more advanced features

---

**Tutorial complete!** 🎉

## Session Information

In [19]:
import sys
import platform

print("Python Session Information")
print("=" * 50)
print(f"Python version: {sys.version}")
print(f"Platform: {platform.platform()}")
print("\nPackage Versions:")
print(f"  navis: {navis.__version__}")
print(f"  pandas: {pd.__version__}")
print(f"  numpy: {np.__version__}")

try:
    import flybrains
    print(f"  flybrains: {flybrains.__version__}")
except:
    print(f"  flybrains: not installed")

print("\n" + "=" * 50)

Python Session Information
Python version: 3.10.19 (main, Oct 21 2025, 16:37:10) [Clang 20.1.8 ]
Platform: macOS-15.6.1-arm64-arm-64bit

Package Versions:
  navis: 1.10.0
  pandas: 2.3.3
  numpy: 2.2.6
  flybrains: not installed

